In [15]:
import pandas as pd
import numpy as np
import numpy.ma as npma
import scipy as sc
from scipy import stats
from scipy.stats import norm, skew #for some statistics
import matplotlib.pyplot as plt
import seaborn as sns
import os
import re
import sklearn as skl
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.kernel_ridge import KernelRidge
from sklearn.compose import ColumnTransformer

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing.data import QuantileTransformer
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder
from scipy.special import boxcox1p

from sklearn.linear_model import ElasticNet, Lasso, LassoLars,  BayesianRidge, LassoLarsIC, Ridge
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error
os.sys.path.append(os.path.pardir)
from src.models.ConvexRegressor import ConvexRegressor
from src.models.ConvexRegressor import convregparam
from src.models.ConvexRegressor import convregconv

#############################################################################################################
project_dir =os.path.pardir
raw_path = os.path.join(project_dir,'data','processed')
train_path = os.path.join(raw_path, 'train.csv')
test_path = os.path.join(raw_path, 'test.csv')
sns.set()
train = pd.read_csv(train_path)
test = pd.read_csv(test_path)
train = train.drop(['Id','Unnamed: 0'],axis = 1)
test = test.drop(['Unnamed: 0'],axis = 1)
y = train['SalePrice'].values
Ids = test['Id'].values
train_x = train.drop('SalePrice',axis = 1).values
test_x = test.drop('Id',axis = 1).values

#################################################################################################################
def LearnAndPredict(x_test,x, y, scaler, solver, ids, q = 0.4, filename = 'temp.csv'):
    yscaler = QuantileTransformer(output_distribution='normal').fit(y.reshape([-1,1]))
    y = yscaler.transform(y.reshape([-1,1])).ravel()
    scaler = scaler.fit(x)
    x = scaler.transform(x)
    solver = solver.fit(x, y)
    x_test = scaler.transform(x_test)
    y = solver.predict(x_test)
    y = yscaler.inverse_transform(y.reshape([-1,1])).ravel()
    df_submission = pd.DataFrame({'Id': ids, 'SalePrice' : y} )
    submission_data_path = os.path.join(os.path.pardir,'data','external')
    submission_file_path = os.path.join(submission_data_path, filename)
    df_submission.to_csv(submission_file_path, index=False)
    #print(solver.best_score_)
    #print(solver.best_params_)
    #sns.distplot(y,fit = norm)
    return solver.coef_

def linear_ml_solve(x, y, scaler, solver, q = 0.4):
    yscaler = QuantileTransformer(output_distribution='normal').fit(y.reshape([-1,1]))
    y = yscaler.transform(y.reshape([-1,1])).ravel()
    x = scaler.fit_transform(x)
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=q, random_state=0)
    solver = solver.fit(x_train, y_train)
    score = solver.score(x_test,y_test)
    print(score)
    y = solver.predict(x_test)
    y = yscaler.inverse_transform(y.reshape([-1,1])).ravel()
    print(solver.best_score_)
    print(solver.best_params_)
    #sns.distplot(y,fit = norm)
    return y

# 1. Lasso and Ridge

In [3]:
scaler = RobustScaler()
est1 = Ridge()
est2 = Lasso()
estimators = [est1,est2]
cest = ConvexRegressor(ests=estimators)
param_grid = convregparam(est2,{'alpha': [ 7.0, 6.0, 3.0, 0.02,0.001, 0.00099, 0.009, 0.0008]})
param_grid.update(convregparam(est1,{'alpha' : [7.0, 6.0, 4.0, 2.0, 0.6,0.5,0.55,0.45]}))
param_grid.update({'convparams' : [convregconv(estimators,[0.11, 0.89]), convregconv(estimators,[0.1,0.9]),
                                   convregconv(estimators,[0.09, 0.91]), convregconv(estimators,[0.05,0.95]),
                                   convregconv(estimators,[0.01,0.99]), convregconv(estimators,[0.001,0.999])]})

gsolver = GridSearchCV(cest, cv=5,param_grid=param_grid)
house_price = LearnAndPredict(test_x,train_x, y, scaler, solver = gsolver, ids = Ids, q = 0.4, filename = 'LassoRidge.csv')
#0.919300261382389
#{'Lasso$$&&$$alpha': 0.001, 'Ridge$$&&$$alpha': 7.0, 'convparams': {'Ridge': 0.001, 'Lasso': 0.999}}

{'Lasso$$&&$$alpha': [7.0, 6.0, 3.0, 0.02, 0.001, 0.00099, 0.009, 0.0008], 'Ridge$$&&$$alpha': [7.0, 6.0, 4.0, 2.0, 0.6, 0.5, 0.55, 0.45], 'convparams': [{'Ridge': 0.11, 'Lasso': 0.89}, {'Ridge': 0.1, 'Lasso': 0.9}, {'Ridge': 0.09, 'Lasso': 0.91}, {'Ridge': 0.05, 'Lasso': 0.95}, {'Ridge': 0.01, 'Lasso': 0.99}, {'Ridge': 0.001, 'Lasso': 0.999}]}
0.919300261382389
{'Lasso$$&&$$alpha': 0.001, 'Ridge$$&&$$alpha': 7.0, 'convparams': {'Ridge': 0.001, 'Lasso': 0.999}}


In [4]:
scaler = RobustScaler()
est1 = Ridge()
est2 = Lasso()
estimators = [est1,est2]
cest = ConvexRegressor(ests=estimators)
param_grid = convregparam(est2,{'alpha': [0.0005, 0.0009, 0.001, 0.0015, 0.0019, 0.002]})
param_grid.update(convregparam(est1,{'alpha' : [7.0, 7.5, 8.0, 8.5, 9.0]}))
param_grid.update({'convparams' : [convregconv(estimators,[0.0001,0.9999]), convregconv(estimators,[0.001,0.999])]})

gsolver = GridSearchCV(cest, cv=5,param_grid=param_grid)
house_price = LearnAndPredict(test_x,train_x, y, scaler, solver = gsolver, ids = Ids, q = 0.4, filename = 'LassoRidge.csv')

0.9193015828986981
{'Lasso$$&&$$alpha': 0.001, 'Ridge$$&&$$alpha': 9.0, 'convparams': {'Ridge': 0.0001, 'Lasso': 0.9999}}


In [ ]:
scaler = RobustScaler()
est = Ridge()
param_grid = {'alpha': [8.0, 7.8, 7.5, 7.3, 7.0]}
gsolver = GridSearchCV(Ridge(max_iter = 10000,tol = 0.00001), cv=5,param_grid=param_grid)
house_price = LearnAndPredict(test_x,train_x, y, scaler, solver = gsolver, ids = Ids, q = 0.4, filename = 'Ridge.csv')

In [16]:
scaler = RobustScaler()
est = Lasso()
param_grid = {'alpha': [5.0,3.0,2.0,1.5,1.3,1.2,1.1,1e0,0.8,0.5,0.2,0.1, 1e-2,0.05,1e-3,0.005, 1e-4,0.0005, 1e-5,0.000051e-6,1e-7]}
param_grid = {'alpha': [ 0.004,0.003, 0.002,0.001, 0.000888889, 0.0008, 0.0005, 0.0009, 0.0015, 0.0019]}
param_grid = {'alpha': [ 0.001]}
gsolver = GridSearchCV(Lasso(max_iter = 10000,tol = 0.0001), cv=5,param_grid=param_grid)
gsolver = Lasso(max_iter = 10000,tol = 0.0001,alpha = 0.001)
house_price = LearnAndPredict(test_x,train_x, y, scaler, solver = gsolver, ids = Ids, q = 0.4, filename = 'Lasso.csv')
#linear_ml_solve(train_x, y, scaler, gsolver, q = 0.4)

In [19]:
(house_price == 0.0).sum()

130